# Temporary Files and Directories

- Automation scripts often need scratch space for intermediate data without cluttering the filesystem or risking name collisions.  
- Hardcoding names like `/tmp/my_file.txt` can lead to security issues, collisions, and manual cleanup.  
- The `tempfile` module provides secure, unique temporary files and directories with optional automatic cleanup. 

## Why Use the tempfile Module?

- It creates files with secure default permissions, preventing unauthorized access on multiuser systems.  
- It generates unique names automatically, avoiding collisions when multiple script instances run concurrently.  
- It integrates with context managers (`with`), enabling automatic cleanup of resources when they're no longer needed.  
- It works across Windows, macOS, and Linux, choosing an appropriate temp location on each platform.  

In [8]:
import tempfile
import os

temp_dir = tempfile.gettempdir()
print(f"Default temporary directory: {temp_dir}")
print(f"Sample contents: {os.listdir(temp_dir)[:5]}")

Default temporary directory: /var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T
Sample contents: ['com.apple.ImageIOXPCService', 'TelemetryUploadFilecom.microsoft.autoupdate.fba.txt', 'com.apple.ThreadCommissionerService', 'com.apple.avconferenced', 'python-languageserver-cancellation']


## tempfile.TemporaryFile()

- Creates an unnamed temporary file opened in binary or text mode.  
- On UNIX-like systems it typically has no name in the filesystem; on Windows it may appear but remains temporary.  
- The file is deleted automatically when closed or when the context block exits.  
- Ideal for internal scratch space that doesn’t need to be passed to external processes.  

In [9]:
import tempfile
from pathlib import Path

with tempfile.TemporaryFile(mode="w+t", encoding="utf-8") as temp_file:
    temp_file.write("This is some temporary data.")
    temp_file.seek(0)
    print("Content from TemporaryFile:")
    print(temp_file.read())


Content from TemporaryFile:
This is some temporary data.


## tempfile.NamedTemporaryFile()

- Creates a temporary file with a visible name in the filesystem.  
- Default `delete=True` removes the file when closed; `delete=False` leaves it for manual cleanup.  
- Use when you need to pass a filename to another process or library.  
- Supports custom `suffix`, `prefix`, and `dir` parameters for naming and placement.  

In [12]:
import tempfile
from pathlib import Path

# Auto-delete on with exit
path = None

with tempfile.NamedTemporaryFile(mode="w+t", encoding="utf-8", suffix=".log") as temp_file:
    path = Path(temp_file.name)
    print(f"Created temp file at {path}. Exists: {path.exists()}")

print(f"After close. Exists? {path.exists()}")

# Persist after with exit
path_persistent = None

with tempfile.NamedTemporaryFile(
    mode="w+t",
    encoding="utf-8",
    suffix=".log",
    delete=False  # keeps temporary file 
) as temp_file:
    path_persistent = Path(temp_file.name)
    print(f"Created temp file at {path}. Exists: {path.exists()}")

print(f"After close. Exists? {path_persistent.exists()}")

if path_persistent.exists():
    path_persistent.unlink()

print(f"After unlink. Exists? {path_persistent.exists()}")

Created temp file at /var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/tmp7pri10gy.log. Exists: True
After close. Exists? False
Created temp file at /var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/tmp7pri10gy.log. Exists: False
After close. Exists? True
After unlink. Exists? False


## tempfile.TemporaryDirectory()

- Creates a new temporary directory, returned as a path string.  
- When used in a `with` block, the directory and everything inside it are deleted on exit.  
- Ideal for workflows that produce multiple temporary files or subdirectories.  

In [11]:
import tempfile
from pathlib import Path

temp_path = None

with tempfile.TemporaryDirectory(prefix="batch_job_") as temp_dir:
    print(f"{temp_dir} - type: {type(temp_dir)}")
    temp_path = Path(temp_dir)
    (temp_path / "file1.txt").write_text("data")
    subdir = temp_path / "subdir"
    subdir.mkdir(exist_ok=True)
    (subdir / "file2.txt").write_text("data2")
    print(f"Contents: {[p.name for p in temp_path.iterdir()]}")

print(f"After close. Exists? {temp_path.exists()}")

/var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/batch_job_va7yrgop - type: <class 'str'>
Contents: ['file1.txt', 'subdir']
After close. Exists? False


## Common Pitfalls & How to Avoid Them

- Calling `os.rmdir()` or `Path.rmdir()` on a non-empty directory raises an error; use `shutil.rmtree()` for recursive deletion.  
- Forgetting to delete files created with `delete=False` in `NamedTemporaryFile` can leave orphaned files.  
- On Windows, other processes can’t open an open temporary file. Use `delete=False` and close it before sharing the name.  
- Relying on a temporary file’s name after closing a `TemporaryFile` is **impossible**, since it may never have had one.  

In [13]:
from pathlib import Path

path = Path('hello.yaml')
print(path.suffix)

.yaml
